# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

**Dataset imported with:** '$ mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json'

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
databases = mongo.list_database_names()
print(databases)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print(collections)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
# review a document in the establishments collection
document = establishments.find_one()
pprint(document)

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
penang_flavors_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant into the collection
result = establishments.insert_one(penang_flavors_data)

In [ ]:
# Check that the new restaurant was inserted
if result:
    print("New restaurant inserted:")
    print(result)
else:
    print("Restaurant not found.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1}
result = establishments.find_one(query, projection)

# Check if the result was found and print the relevant fields
if result:
    print("BusinessTypeID and BusinessType:")
    print(result)
else:
    print("Business type not found.")

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {"BusinessName": "Penang Flavours"} 

# Assuming you have found the BusinessTypeID for "Restaurant/Cafe/Canteen"
business_type_id = "6775d3fd8e548b0163fc842f" 

# Specify the update operation
update_operation = {"$set": {"BusinessTypeID": business_type_id}}

# Update the restaurant
result = establishments.update_one(filter_query, update_operation)

In [ ]:
# Confirm that the new restaurant was updated
if result.modified_count > 0:
    print(f"Penang Flavors updated successfully with BusinessTypeID {business_type_id}")
else:
    print("No changes made or restaurant not found.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_query_delete = {"LocalAuthorityName": "Dover"}
result = establishments.delete_many(dover_query_delete)

In [ ]:
# Check if any remaining documents include Dover
print(f"Number of remaining documents with LocalAuthorityName 'Dover': {result.deleted_count}")

In [ ]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
if remaining_document:
    print("A document remains in the collection:")
    print(remaining_document)
else:
    print("No documents found in the collection.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many(
    {},
    [
        {
            '$set': {
                'latitude': {'$toDecimal': '$latitude'},
                'longitude': {'$toDecimal': '$longitude'}
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many(
    {},
    [
        {
            '$set': {
                'RatingValue': {'$toInt': '$RatingValue'}
            }
        }
    ]
)

In [ ]:
# Check that the coordinates and rating value are now numbers
document = establishments.find_one()

# Check if a document was found and print the relevant fields and their types
if document:
    print("Document details:")
    print(f"Latitude: {document['latitude']} (Type: {type(document['latitude']).__name__})")
    print(f"Longitude: {document['longitude']} (Type: {type(document['longitude']).__name__})")
    print(f"RatingValue: {document['RatingValue']} (Type: {type(document['RatingValue']).__name__})")
else:
    print("No documents found in the collection.")